In [2]:
pip install GoogleNews

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 5.6 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [15]:
from tpot import TPOTClassifier
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import get_scorer_names, get_scorer
import time
from GoogleNews import GoogleNews
from datetime import datetime, timedelta

In [23]:
company_data = pd.read_csv('companyListURL.tsv', delimiter='\t',encoding='utf-8')
first_two_rows = company_data.head(2)
first_two_rows.head()

,sNo,Company Name,URL
0,1,ABFRL,https://www.tickertape.in/stocks/aditya-birla-...
1,2,ACC,https://www.tickertape.in/stocks/acc-ACC


In [24]:
company_name_all = first_two_rows['Company Name']
all_results = []  # Create an empty list to store the results

for company_name in company_name_all:
    if company_name != '':
        googlenews = GoogleNews()
        googlenews.search(company_name)
        end_date = datetime.now()
        start_date = end_date - timedelta(hours=1)  # Select news from the last 1 hour
        start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
        end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')
        googlenews.set_time_range(start_date_str, end_date_str)
        googlenews.search(company_name)
        result = googlenews.result()
        total_count = googlenews.total_count()
        print(total_count)
        company_link_df = pd.DataFrame(result[:100])  # Limit to 100 articles per company
        all_results.append(company_link_df)  # Append the results to the list

combined_results = pd.concat(all_results)  # Concatenate all the results into a single DataFrame
print(combined_results)

HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
0
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
0
Empty DataFrame
Columns: []
Index: []


In [25]:
try:
    list =[] #creating an empty list 
    for i in combined_results.index:
        dict = {} #creating an empty dictionary to append an article in every single iteration
        news_article = Article(combined_results['link'][i],config=config) #providing the link
        try:
          news_article.download()
          news_article.parse()
          news_article.nlp()
        except:
           pass 
        
        dict['NewsDate']=combined_results['datetime'][i]
        dict['NewsTitle']=news_article.title
        dict['NewsArticle']=news_article.text
        dict['Summary']=news_article.summary
        list.append(dict)
    empty = not any(list)
    
    if empty == False:
        news_article_df=pd.DataFrame(list)
        print(news_article_df)

except Exception as e:
    #exception handling
    print("error occurred:" + str(e))
    print('----------' )

In [ ]:
def percentage(part,whole):
    return 100 * float(part)/float(whole)

positive = 0
negative = 0
neutral = 0
newssentiment={}

for news_article in news_article_df['Summary']:
    analyzer = SentimentIntensityAnalyzer().polarity_scores(news)
    negative = analyzer['neg']
    neutral = analyzer['neu']
    positive = analyzer['pos']
    compound = analyzer['compound']
    print(neg,pos,neu,comp)
    newssentiment.setdefault('Negative',[]).append(negative)
    newssentiment.setdefault('Positive',[]).append(positive)
    newssentiment.setdefault('Neutral',[]).append(neutral)
    newssentiment.setdefault('Comp',[]).append(compound)
NewsSentimentDF=pd.DataFrame(newssentiment)
NewsSentimentDF

In [ ]:
NewsArticleSentimentDF=pd.concat([news_article_df,NewsSentimentDF],axis=1)
NewsArticleSentimentDF

In [ ]:
NewsArticleSentimentDF.to_excel('NewsArticleSentiment.xlsx')